In [1]:
import json
import os
import typing
import datetime

PARENT_DIR = os.path.dirname(os.getcwd())

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive import interactive_runner

In [20]:
class EventLog(typing.NamedTuple):
    ip: str
    id: str
    lat: float
    lng: float
    user_agent: str
    age_bracket: str
    opted_into_marketing: bool
    http_request: str
    http_response: int
    file_size_bytes: int
    event_datetime: str
    event_ts: int

class EventTS(typing.NamedTuple):
    event_ts: datetime.datetime

beam.coders.registry.register_coder(EventLog, beam.coders.RowCoder)

def parse_json(element: str):
    row = json.loads(element)
    # lat/lng sometimes empty string
    if not row["lat"] or not row["lng"]:
        row = {**row, **{"lat": -1, "lng": -1}}
    return EventLog(**row)

def select_ts(element: EventLog):
    ts = datetime.datetime.strptime(element.event_datetime, "%Y-%m-%dT%H:%M:%S.%f")
    return EventTS(event_ts=ts)
    # return EventTS(event_ts=datetime.datetime.strftime(ts, "%Y-%m-%d %H:%M:%S"))


In [21]:
p = beam.Pipeline(interactive_runner.InteractiveRunner())

events = (
    p
    | "Read from files"
    >> beam.io.ReadFromText(
        file_pattern=os.path.join(os.path.join(PARENT_DIR, "inputs", "*.out"))
    )
    | "Parse elements" >> beam.Map(parse_json).with_output_types(EventLog)
    | "Format timestamp" >> beam.Map(select_ts).with_output_types(EventTS)
)

ib.show(events, n=5)

In [8]:
%load_ext apache_beam.runners.interactive.sql.beam_sql_magics

The apache_beam.runners.interactive.sql.beam_sql_magics extension is already loaded. To reload it, use:
  %reload_ext apache_beam.runners.interactive.sql.beam_sql_magics


In [22]:
%%beam_sql -o output
SELECT event_ts FROM events limit 5

usage: beam_sql magic to execute Beam SQL in notebooks
---------------------------------------------------------
%beam_sql [-o OUTPUT_NAME] [-v] [-r RUNNER] query
---------------------------------------------------------
Or
---------------------------------------------------------
%%beam_sql [-o OUTPUT_NAME] [-v] [-r RUNNER] query-line#1
query-line#2
...
query-line#N
---------------------------------------------------------

positional arguments:
  query                 The Beam SQL query to execute. Syntax: https://beam.ap
                        ache.org/documentation/dsls/sql/calcite/query-syntax/.
                        Please make sure that there is no conflict between
                        your variable names and the SQL keywords, such as
                        "SELECT", "FROM", "WHERE" and etc.

options:
  -h, --help            show this help message and exit
  -o OUTPUT_NAME, --output-name OUTPUT_NAME
                        The output variable name of the magic, usually a


RuntimeError: org.apache.beam.vendor.guava.v32_1_2_jre.com.google.common.util.concurrent.UncheckedExecutionException: org.apache.beam.vendor.guava.v32_1_2_jre.com.google.common.util.concurrent.UncheckedExecutionException: java.lang.IllegalArgumentException: Failed to decode Schema due to an error decoding Field proto:

name: "event_ts"
type {
  nullable: true
  logical_type {
    urn: "beam:logical:pythonsdk_any:v1"
  }
}

	at org.apache.beam.vendor.guava.v32_1_2_jre.com.google.common.cache.LocalCache$Segment.get(LocalCache.java:2086)
	at org.apache.beam.vendor.guava.v32_1_2_jre.com.google.common.cache.LocalCache.get(LocalCache.java:4012)
	at org.apache.beam.vendor.guava.v32_1_2_jre.com.google.common.cache.LocalCache.getOrLoad(LocalCache.java:4035)
	at org.apache.beam.vendor.guava.v32_1_2_jre.com.google.common.cache.LocalCache$LocalLoadingCache.get(LocalCache.java:5013)
	at org.apache.beam.runners.core.construction.RehydratedComponents.getPCollection(RehydratedComponents.java:139)
	at 